In [1]:
import mlflow
from lung_function.modules.compute_metrics import icc, metrics
from mlflow import log_params
import pandas as pd
from pathlib import Path
import numpy as np 
import pandas as pd
from mlflow import MlflowClient


/home/jjia/.local/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/jjia/.local/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [15]:
parent_ids = [
902,
904,
906,
908,
910,
912,
2521,
2527,
2529,
2532,
2534,
2536,
2538,
2540,
2542,
2568,
2570,
2572,
2574,
2576,
2594,
2600,
2605,
2607,
2258,
2140,
2138,
2148,
2144,
2142,
2194,
2667,
2665,
2663,
2661,
2655,
2654,
2651,
2649,
2647,
2645,
2643,
2714,
2712,
2710,
2740,
2742,
2732,
2733,
2663,
2654,
2655,
2661,
2758,
2756,
2754,
2752,
2750,
2731,
2730,
2720,
2859,
2860,
2856,
2855,
2828,
2827,
2813,
2814,
2809,
2810,
2806,
2805,
2803,
2801,
2799,
2797,
2783,
2780,
2779,
2777,
2775,
2758,
]
parent_ids = [1303, 1308, 1458, 1460, 1503, 1513, 1516, 1526, 1538,1560,1577,1579,1585,1608,1610,1612,1614,1616,1633,1635,1637,1639,1641,1648,1650,1652,1654,1656,1676,1678,1680,1682,1684,1687,1710,1714,1725,1753,1755,1757,1786,1788,1790,1798,1800,1802,1812,1816,1840,1842,1844,1846,1848,1867,1865,1869,1882,1884,1896,1898,1908,1910,1918,1927,1930,1932,1934,2203,2228,2321,2323,2325,2327,2329,2335,2339,2331,2353,2362,2364,2366,2368,2370,2372,2408,2410,2432,2440,2450,2456,2460,2462,2464,2466,2468,2484,2486,2488]
# parent_ids = [2758]
parent_ids = list(set(parent_ids))
parent_ids.sort()

In [2]:
parent_ids = [2750, 2731]

In [9]:
len(parent_ids)

98

In [10]:
def child_run_id(parent_id: str, experiment, client):
    
    parent_run = client.search_runs(experiment_ids=[experiment.experiment_id], filter_string=f"params.id = '{str(parent_id)}'")[0]
    runss = client.search_runs(experiment_ids=[experiment.experiment_id], filter_string=f"tags.mlflow.parentRunId = '{parent_run.info.run_id}'")  # 4 days after after the parent ID created 
    
    if len(runss) == 4:  # all folds are there
        out = {int(ru.data.params['fold']): int(ru.data.params['id']) for ru in runss}
        out[0 ] = parent_id
        print(f"successfully found four folds runs for {parent_id}: {out}")
        return out
    else:
        tmp = {int(ru.data.params['fold']): int(ru.data.params['id']) for ru in runss}
        print(f"the searched runs for {parent_id} do not 4 folds, only has {len(runss)} folds: {tmp}")
        return None
    
     

In [16]:
mlflow.set_tracking_uri("http://nodelogin02:5000")
experiment = mlflow.set_experiment("lung_fun_db15")
client = MlflowClient()

fold_ex_dt_ls = [ ]
for parent_id in parent_ids[::-1]:
    dt = child_run_id(parent_id, experiment, client)
    if dt:
        fold_ex_dt_ls.append(dt)


successfully found four folds runs for 2488: {4: 2520, 3: 2511, 2: 2502, 1: 2489, 0: 2488}
successfully found four folds runs for 2486: {4: 2519, 3: 2510, 2: 2501, 1: 2487, 0: 2486}
successfully found four folds runs for 2484: {4: 2518, 3: 2509, 2: 2500, 1: 2485, 0: 2484}
successfully found four folds runs for 2468: {4: 2517, 3: 2508, 2: 2499, 1: 2469, 0: 2468}
successfully found four folds runs for 2466: {4: 2515, 3: 2506, 2: 2498, 1: 2467, 0: 2466}
successfully found four folds runs for 2464: {4: 2516, 3: 2507, 2: 2497, 1: 2465, 0: 2464}
successfully found four folds runs for 2462: {4: 2514, 3: 2505, 2: 2496, 1: 2463, 0: 2462}
successfully found four folds runs for 2460: {4: 2513, 3: 2504, 2: 2495, 1: 2461, 0: 2460}
successfully found four folds runs for 2456: {4: 2512, 3: 2503, 2: 2494, 1: 2457, 0: 2456}
the searched runs for 2450 do not 4 folds, only has 1 folds: {1: 2451}
the searched runs for 2440 do not 4 folds, only has 1 folds: {1: 2441}
the searched runs for 2432 do not 4 fol

## merging 4 folds

In [17]:
def ensemble_4folds_testing(fold_ex_dt_ls):
    parent_dir = '/home/jjia/data/lung_function/lung_function/scripts/results/experiments/'

    for fold_ex_dt in fold_ex_dt_ls:
        dir0 = parent_dir + str(fold_ex_dt[0])
        label_fpath = dir0  + '/test_label.csv'
        ave_fpath =dir0  + '/test_pred.csv'
        output_file_path = Path(ave_fpath)
        output_file_path.parent.mkdir(parents=True, exist_ok=True)
        
        df_ls = []
        for i in [1,2,3,4]:
            data_fpath = parent_dir + str(fold_ex_dt[i]) + '/test_pred.csv'
            df = pd.read_csv(data_fpath,index_col=0)
            df_ls.append(df)
        df_ave = (df_ls[0] + df_ls[1] + df_ls[2] + df_ls[3])/4
        df_ave.to_csv(ave_fpath)
        
        label_fpath_fold1 = parent_dir + str(fold_ex_dt[i]) + '/test_label.csv'
        df_label = pd.read_csv(label_fpath_fold1,index_col=0)
        df_label.to_csv(label_fpath)
        
        print(ave_fpath)

def ensemble_4folds_validation(fold_ex_dt_ls):
    parent_dir = '/home/jjia/data/lung_function/lung_function/scripts/results/experiments/'

    for fold_ex_dt in fold_ex_dt_ls:
        dir0 = parent_dir + str(fold_ex_dt[0])
        pred_all_fpath = dir0  + '/valid_pred.csv'
        label_all_fpath = dir0  + '/valid_label.csv'
        output_file_path = Path(pred_all_fpath)
        output_file_path.parent.mkdir(parents=True, exist_ok=True)
        
        df_pred_ls, df_label_ls = [], []
        for i in [1,2,3,4]:
            data_fpath = parent_dir + str(fold_ex_dt[i]) + '/valid_pred.csv'
            label_fpath = parent_dir + str(fold_ex_dt[i]) + '/valid_label.csv'
            df_pred = pd.read_csv(data_fpath,index_col=0)
            df_label = pd.read_csv(label_fpath,index_col=0)

            df_pred_ls.append(df_pred)
            df_label_ls.append(df_label)
        df_pred_valid = pd.concat(df_pred_ls)
        df_label_valid = pd.concat(df_label_ls)
        
        df_pred_valid.to_csv(pred_all_fpath)
        df_label_valid.to_csv(label_all_fpath)
        print(pred_all_fpath)

        
        
        

In [18]:

ensemble_4folds_testing(fold_ex_dt_ls)
ensemble_4folds_validation(fold_ex_dt_ls)

/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2488/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2486/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2484/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2468/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2466/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2464/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2462/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2460/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2456/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2410/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2408/test_pred.csv

## Calculate relative error

In [37]:
# calculate the standard reference
parent_dir = '/home/jjia/data/lung_function/lung_function/scripts/results/experiments/'
for i in fold_ex_dt_ls:
    label_fpath = f"{parent_dir}{i[1]}/{mode}_label.csv"
    label = pd.read_csv(label_fpath).iloc[: , 1:]
    label_pp = (0.15 / label).mean() * 100
    print(label.columns, label_pp)


Index(['DLCOc_SB'], dtype='object') DLCOc_SB    2.78599
dtype: float64
Index(['TLC_He'], dtype='object') TLC_He    3.000312
dtype: float64
Index(['FVC'], dtype='object') FVC    4.470441
dtype: float64
Index(['FEV1'], dtype='object') FEV1    5.781361
dtype: float64


## adding parameters

In [19]:
def mae(pred_fpath, label_fpath, ignore_1st_column=True):
    mae_dict = {}

    label = pd.read_csv(label_fpath)
    pred = pd.read_csv(pred_fpath)
    if ignore_1st_column:
        pred = pred.iloc[: , 1:]
        label = label.iloc[: , 1:]
    if 'ID' == label.columns[0]:
        del label["ID"]
    if 'ID' == pred.columns[0]:
        del pred["ID"]

    original_columns = label.columns

    # ori_columns = list(label.columns)

    for column in original_columns:
        abs_err = (pred[column] - label[column]).abs()
        mae_value = abs_err.mean().round(2)
        std_value = abs_err.std().round(2)
        
        prefix = label_fpath.split("/")[-1].split("_")[0]
        mae_dict['mae_' + prefix + '_' + column] = mae_value
        mae_dict['mae_std_' + prefix + '_' + column] = std_value

    return mae_dict

def me(pred_fpath, label_fpath, ignore_1st_column=True):
    mae_dict = {}

    label = pd.read_csv(label_fpath)
    pred = pd.read_csv(pred_fpath)
    if ignore_1st_column:
        pred = pred.iloc[: , 1:]
        label = label.iloc[: , 1:]
    if 'ID' == label.columns[0]:
        del label["ID"]
    if 'ID' == pred.columns[0]:
        del pred["ID"]

    original_columns = label.columns

    for column in original_columns:
        abs_err = (pred[column] - label[column])
        mae_value = abs_err.mean().round(2)
        std_value = abs_err.std().round(2)
        
        prefix = label_fpath.split("/")[-1].split("_")[0]
        mae_dict['me_' + prefix + '_' + column] = mae_value
        mae_dict['me_std_' + prefix + '_' + column] = std_value

    return mae_dict

def mre(pred_fpath, label_fpath, ignore_1st_column=True):
    label = pd.read_csv(label_fpath)
    pred = pd.read_csv(pred_fpath)
    
    if ignore_1st_column:
        pred = pred.iloc[: , 1:]
        label = label.iloc[: , 1:]

    rel_err_dict = {}
    for column in label.columns:
        mae_value = (pred[column] - label[column]).abs()
        rel_err = mae_value / label[column]
        # print(f'relative error for {column}:')
        # for i in rel_err:
        #     if i > 2:
        #         print(i)
        mean_rel_err = rel_err.mean().round(2)
        mean_rel_err_std = rel_err.std().round(2)
        prefix = label_fpath.split("/")[-1].split("_")[0]
        rel_err_dict['mre_' + prefix + '_' + column] = mean_rel_err
        rel_err_dict['mre_std_' + prefix + '_' + column] = mean_rel_err_std
       
    return rel_err_dict

In [20]:
mlflow.set_tracking_uri("http://nodelogin02:5000")
experiment = mlflow.set_experiment("lung_fun_db15")

# Create a run under the default experiment (whose id is '0').
client = mlflow.MlflowClient()
parent_dir = '/home/jjia/data/lung_function/lung_function/scripts/results/experiments/'

for i in fold_ex_dt_ls:
    print(i)
    experiment_id = i[0]
    run_ = client.search_runs(experiment_ids=[experiment.experiment_id], filter_string=f"params.id = '{str(experiment_id)}'")[0]
    run_id = run_.info.run_id
    
    with mlflow.start_run(run_id=run_id):
        for mode in ['valid', 'test']:
            print(mode)
            label_fpath = f"{parent_dir}{i[0]}/{mode}_label.csv"
            pred_fpath = f"{parent_dir}{i[0]}/{mode}_pred.csv"

            # #add icc
            icc_value = icc(label_fpath, pred_fpath, ignore_1st_column=True)
            icc_value_ensemble = {'ensemble_' + k:v  for k, v in icc_value.items()}  # update keys
            print(icc_value_ensemble)
            try:
                log_params(icc_value_ensemble)
            except:
                pass

            # add r
            r_p_value = metrics(pred_fpath, label_fpath, ignore_1st_column=True)
            r_p_value_ensemble = {'ensemble_' + k:v  for k, v in r_p_value.items()}  # update keys
            print(r_p_value_ensemble)
            try:
                log_params(r_p_value_ensemble)
            except:
                pass
            
            # add mae
            mae_dict = mae(pred_fpath, label_fpath, ignore_1st_column=True)
            mae_ensemble = {'ensemble_' + k:v for k, v in mae_dict.items()}
            print(mae_ensemble)
            try:
                log_params(mae_ensemble)
            except:
                pass
            
            # add me
            me_dict = me(pred_fpath, label_fpath, ignore_1st_column=True)
            me_ensemble = {'ensemble_' + k:v for k, v in me_dict.items()}
            print(me_ensemble)
            try:
                log_params(me_ensemble)
            except:
                pass   
            
            # add mre
            mre_dict = mre(pred_fpath, label_fpath, ignore_1st_column=True)
            mre_ensemble = {'ensemble_' + k:v for k, v in mre_dict.items()}
            print(mre_ensemble)
            try:
                log_params(mre_ensemble)
            except:
                pass   

{4: 2520, 3: 2511, 2: 2502, 1: 2489, 0: 2488}
valid
{'ensemble_icc_valid_FVC': 0.49, 'ensemble_icc_valid_DLCOc_SB': 0.43, 'ensemble_icc_valid_FEV1': 0.46, 'ensemble_icc_valid_TLC_He': 0.47}
len_df_label 253
['FVC', 'DLCOc', 'FEV1', 'TLC'] yyaayy


TypeError: mean_diff_plot() got an unexpected keyword argument 'class_name'

In [6]:
import importlib
importlib.reload(lung_function.modules.compute_metrics.metrics)

NameError: name 'lung_function' is not defined